In [30]:
using JLD2
using Flux
using Statistics
using ProgressMeter

In [31]:
@load "Training.jld2" inputs;


In [32]:
@load "Targets.jld2"  actuals;

In [45]:
input = inputs[2,:];
#input = reshape(input,:,100);
input

50000-element Vector{Float64}:
 2.3908511486970712e-9
 2.436013828157643e-9
 2.488448322654356e-9
 2.5461683866947014e-9
 2.6061683790637417e-9
 2.664820124211142e-9
 2.718154819959127e-9
 2.7622417315175367e-9
 2.7933285545808926e-9
 2.8080158022726244e-9
 ⋮
 4.0714169026759647e-13
 3.7295985458878003e-13
 3.366960978134348e-13
 2.8178885545304874e-13
 2.1419920234758155e-13
 1.4220594657073096e-13
 8.898707432537348e-14
 5.6576979018973334e-14
 2.1029735140690506e-14

In [34]:
idx = rand(1:length(inputs[:,1]))
testSet = inputs[idx,:];
testSet = reshape(testSet,:,100);
testTarg = actuals[idx]

538100.0

In [35]:
actual = actuals[1];

In [36]:
model = Chain(
    RNN(100,1, NNlib.relu)
) |> gpu;

In [37]:
Flux.reset!(model)

In [38]:
# out1=-1
# for i in eachrow(testSet)
#     out1 = model(i |> gpu) |> cpu
# end
loss(testSet, testTarg)

2.8955161e11

In [39]:
Flux.reset!(model)
function eval_model(x)
    out1=-1
    for i in eachrow(x)
        out1 = model(i |> gpu) |> cpu
    end
    out1
end

loss(x, y) = Flux.Losses.mse(eval_model(x), y)

ps = Flux.params(model)

opt = Flux.ADAM();

evalcb() = @show(sum(loss.(testSet, testTarg)))

evalcb (generic function with 1 method)

In [43]:
Flux.train!(loss, ps, zip(inputs,actuals), opt)


MethodError: MethodError: no method matching eachrow(::Float64)
Closest candidates are:
  eachrow(!Matched::AbstractVecOrMat) at abstractarraymath.jl:550

In [41]:
# num_epochs = length(inputs[:,1])
# for x in 1:num_epochs
#     Flux.train!(loss, ps, zip(inputs[x,:],actuals[x]), opt, cb = Flux.throttle(evalcb, 1))
# end

In [42]:
println("Test loss after = ", sum(loss.(test_data, test_labels)))

UndefVarError: UndefVarError: test_data not defined